In [7]:
import pandas as pd
import numpy as np
import config
import time
from numba import jit

In [8]:
def timestamp_datetime(value):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(value))

def time_feat(df,featList,featName):
    '''scaler = StandardScaler()
    tmp = df.groupby(featList).size().reset_index().rename(columns={0:featName})
    tmp[featName] = scaler.fit_transform(tmp[featName].values.reshape(-1,1))
    df = df.merge(tmp,'left',on=featList)'''
    df[featName] = df.groupby(featList)['context_timestamp'].rank(method='first')   
    return df

def process(df):
    df['time'] = df.context_timestamp.apply(timestamp_datetime)
    df['day'] = df.time.apply(lambda x: int(x[8:10]))
    df['hour'] = df.time.apply(lambda x: int(x[11:13]))
    '''for lst in timeFeatList:
        df = time_feat(df,lst,'_'.join(lst))'''
    df['item_property_list_clean'] = df['item_property_list'].apply(lambda x:';'.join(sorted(set(str(x).split(';')))))
    for i in range(3):
        df['item_category_list_bin%d'%i] = df['item_category_list'].apply(lambda x: x.split(';')[i] if len(x.split(';'))>i else -1)
    df["missing_feat"] = np.sum((df == -1).values, axis=1)
    return df

In [9]:
def focus_one_record(df,key_var=[]):
    nameBase = '_'.join(key_var)
    ###当天前后的数据情况
    df[nameBase+'_before_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min') - 1
    df[nameBase+'_after_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min',ascending=False)- 1
    df[nameBase+'_sametime_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='max') - df.groupby(key_var+['day'])['context_timestamp'].rank(method='min')+1
    df = df.merge(df.groupby(key_var+['day'],as_index=False)['context_timestamp'].agg({nameBase+'_day_cnt':'count'}),'inner',key_var+['day'])
    for feat in ['_before_cnt','_after_cnt','_sametime_cnt']:
        df[nameBase+feat+'_ratio'] = df[nameBase+feat]*1.0/df[nameBase+'_day_cnt']
        
    ###前一天购买/浏览的数量
    '''dfTmp =  df.groupby(key_var+['day'],as_index=False)['is_trade'].agg({nameBase+'_preday_trade_cnt':'sum',nameBase+'_preday_cnt':'count'})
    dfTmp['day'] = dfTmp['day']+1
    df = df.merge(dfTmp,'left',key_var+['day'])
    df[nameBase+'_preday_trade_ratio'] = df[nameBase+'_preday_trade_cnt']*1.0/df[nameBase+'_preday_cnt']
    for feat in ['_preday_trade_cnt','_preday_cnt','_preday_trade_ratio']:
        df[nameBase+feat].fillna(0,inplace=True)'''
        
    ###广告展示上下间隔
    dfTmp = df[[nameBase+'_before_cnt',nameBase+'_after_cnt',nameBase+'_sametime_cnt','time']+key_var+['day']]
    dfTmp.rename(columns={'time':'new_time'},inplace=True)
    dfTmp['next_record'] = dfTmp[nameBase+'_before_cnt'] + dfTmp[nameBase+'_sametime_cnt'] + 1
    dfTmp['last_record'] = dfTmp[nameBase+'_after_cnt'] + dfTmp[nameBase+'_sametime_cnt'] + 1
    df = df.merge(dfTmp[key_var+['day','next_record','new_time']],'left',left_on = key_var+['day',nameBase+'_before_cnt'],right_on = key_var+['day','next_record'])
    df['next_time_dur'] = (pd.to_datetime(df['time'])-pd.to_datetime(df['new_time'])).dt.seconds
    df['next_time_dur'].fillna(999999,inplace=True)
    df.loc[df[nameBase+'_sametime_cnt']>1,'next_time_dur'] = 0
    del df['new_time']
    del df['next_record']
    
    df = df.merge(dfTmp[key_var+['day','last_record','new_time']],'left',left_on = key_var+['day',nameBase+'_after_cnt'],right_on = key_var+['day','last_record'])
    df['last_time_dur'] = (pd.to_datetime(df['new_time'])-pd.to_datetime(df['time'])).dt.seconds
    df['last_time_dur'].fillna(999999,inplace=True)
    df.loc[df[nameBase+'_sametime_cnt']>1,'last_time_dur'] = 0
    del df['new_time']
    del df['last_record']
    return df    
    

In [10]:
def Discretize(vlu, list_cat, greedy_f=False):
    if len(list_cat) == 0:
        return set([str(vlu)])
    cut_l = ['-Inf'] + [str(i) for i in list_cat if i < vlu]
    cut_h = [str(i) for i in list_cat if i >= vlu] + ['Inf']
    if greedy_f:
        return set([i + '_' + j for i in cut_l for j in cut_h])
    else:
        return set([cut_l[-1] + '_' + cut_h[0]])
    
def Discretization(rec, dict_cat, greedy_f=False):
    return (dict([(key, Discretize(rec[key], dict_cat[key], greedy_f)) for key in dict_cat]))

def cat_str(rec, dict_cat, greedy_f=False):
    cat_tag = Discretization(rec, dict_cat, greedy_f=False)
    flat_list = ';'.join([key + '_' + str(item) for key in cat_tag.keys() for item in cat_tag[key]])
    return (flat_list)

@jit
def _agg_df(df, grp_key, sum_var, cnt_var, stat_var):
    grouped = df.groupby(grp_key)
    agg_sum = grouped[sum_var].agg('sum').reset_index().melt(id_vars=grp_key, value_vars=sum_var)
    agg_sum['method'] = 'sum'
    agg_uniq = grouped[cnt_var].agg('nunique').reset_index().melt(id_vars=grp_key, value_vars=cnt_var)
    agg_uniq['method'] = 'cnt'
    agg_max = grouped[stat_var].agg('max').reset_index().melt(id_vars=grp_key, value_vars=stat_var)
    agg_max['method'] = 'max'
    agg_min = grouped[stat_var].agg('min').reset_index().melt(id_vars=grp_key, value_vars=stat_var)
    agg_min['method'] = 'min'
    agg_mst = pd.concat([agg_sum, agg_uniq, agg_max, agg_min])
    return (agg_mst)

def _sig_calc(cdr_rec, key_var, TS_cat, dict_cat,sum_var, cnt_var, stat_var):
    cdr_dict = cdr_rec.to_dict(orient='records')
    cdr_dict = [dict(list(x.items()) + [('TS', '~'.join(list(Discretization(x, TS_cat).values())[0]))] + [
        ('cat_str', cat_str(x, dict_cat))]) for x in cdr_dict]
    cdr_rec = pd.DataFrame(cdr_dict).reset_index()
    # one to many
    cat_driver = pd.concat([pd.Series(row['index'], row['cat_str'].split(';'))
                            for _, row in cdr_rec.iterrows()]).reset_index()
    cat_driver.columns = ['cat', 'index']
    cdr_rec_m = pd.merge(cdr_rec, cat_driver, on='index', how='inner')
    # Aggregation
    if key_var=='':
        key_var_tmp = ['TS']
    else:
        key_var_tmp = key_var+['TS']
    agg_mst_m = _agg_df(cdr_rec_m, key_var_tmp+['cat'], sum_var, cnt_var, stat_var)
    agg_mst_m['var_name'] = agg_mst_m[['method', 'variable', 'cat']].apply(
        lambda x: x[0] + '_' + x[1] + '_cat_' + str(x[2]), axis=1)
    if key_var=='':
        agg_mst_m['key'] = agg_mst_m[key_var_tmp]
    else:
        agg_mst_m['key'] = agg_mst_m[key_var_tmp].apply(lambda x: '{0}:{1}'.format(x[0], x[1]), axis=1)
    agg_df_m = agg_mst_m.pivot(index='key', columns='var_name', values='value').reset_index()

    agg_mst = _agg_df(cdr_rec, key_var_tmp, sum_var, cnt_var, stat_var)
    agg_mst['var_name'] = agg_mst[['method', 'variable']].apply(lambda x: x[0] + '_' + x[1] + '_cat_total_all', axis=1)
    if key_var=='':
        agg_mst['key'] = agg_mst[key_var_tmp]
    else:
        agg_mst['key'] = agg_mst[key_var_tmp].apply(lambda x: '{0}:{1}'.format(x[0], x[1]), axis=1)
    agg_df = agg_mst.pivot(index='key', columns='var_name', values='value').reset_index()

    agg_df = pd.merge(agg_df, agg_df_m, on='key', how='left')
    
    sig_ratio = [sig for sig in agg_df.columns if
                 (('sum_' in sig) or ('cnt_' in sig)) and ('cat_total_all' not in sig)]
    for sig in sig_ratio:
        agg_df['rto_' + sig] = agg_df[[sig, sig.split('cat')[0] + 'cat_total_all']].apply(
            lambda x: (x[0] + 0.0) / x[1], axis=1)
        del agg_df[sig]
    return agg_df

In [11]:
def group_count(cdr_rec, key_var,cal_var,count_var = 'instance_id',ratio = True):
    base_name = '~'.join(key_var)
    dfGroup = cdr_rec.groupby(key_var,as_index=False)[count_var].agg({base_name+'_cnt': 'count'})
    cdr_rec = cdr_rec.merge(dfGroup,on=key_var,how='left')
    for var in cal_var:
        if var in key_var:
            continue
        dfGroup = cdr_rec.groupby(key_var+[var],as_index=False)[count_var].agg({str(var) + '_'+base_name+'_cnt': 'count'})
        cdr_rec = cdr_rec.merge(dfGroup,on=key_var+[var],how='left')
        if ratio:
            cdr_rec[str(var) + '_'+base_name+'_ratio'] = 1.0*cdr_rec[str(var) + '_'+base_name+'_cnt']/cdr_rec[base_name+'_cnt']
            del cdr_rec[str(var) + '_'+base_name+'_cnt']
    del cdr_rec[base_name+'_cnt']
    return cdr_rec
def group_unique(cdr_rec, key_var,cal_var,ratio = True):
    for var in cal_var:
        if var in key_var:
            continue
        dfGroup = cdr_rec.groupby(key_var,as_index=False)[var].agg({str(var) + '_'+base_name+'_nunique': 'nunique'})
        cdr_rec = cdr_rec.merge(dfGroup,on=key_var,how='left')
    return cdr_rec

In [12]:
dfTrain = pd.read_table(config.TRAIN_FILE,sep=' ')
dfTrain.drop_duplicates(inplace=True)
dfTrain.reset_index(inplace=True,drop =True)
dfTest = pd.read_table(config.TEST_FILE,sep=' ')

dfTrain = process(dfTrain)
dfTest = process(dfTest)

dfAll = pd.concat([dfTrain,dfTest],axis=0)
dfAll.reset_index(inplace=True,drop=True)
trainNum = dfTrain.shape[0]
dfAll['cnt_rec'] = 1
dfAll.shape

(496482, 36)

In [ ]:
#userid相关表现
userid_total = dfAll.groupby(['user_id'])['instance_id'].count()
userid_trade = dfAll.groupby(['user_id'])['is_trade'].sum()
userid_notrade = userid_total - userid_trade
userid_trade_rate = userid_trade*1.0/userid_total



In [ ]:
for varKey in ['use_id','shop_id','item_id']: 
    tmpFeat = _sig_calc(daAll,[varKey],{'day':[18,19,20,21,22,23,24,25]},{'item_sales_level':[],'item_collected_level':[]},['user_gender_id'],['cnt_ind'],['cnt_ind'])

In [13]:
sum_var = ['cnt_rec']
cnt_var = ['cnt_rec']
stat_var = ['cnt_rec']

In [14]:
dict_cat = {'cnt_rec':[],'item_price_level':[],'item_sales_level':[],'item_collected_level':[],'item_pv_level':[],'hour':[6,12,20]}
TS_cat = {'day':[18,19,20,21,22,23,24,25]}
key_var = ['user_id']

In [15]:
_sig_calc(dfAll,['user_id'],{'day':[18,19,20,21,22,23,24,25]},dict_cat,sum_var,cnt_var,stat_var)

var_name,key,cnt_cnt_rec_cat_total_all,max_cnt_rec_cat_total_all,min_cnt_rec_cat_total_all,sum_cnt_rec_cat_total_all,rto_cnt_cnt_rec_cat_cnt_rec_1,rto_cnt_cnt_rec_cat_hour_-Inf_6,rto_cnt_cnt_rec_cat_hour_12_20,rto_cnt_cnt_rec_cat_hour_20_Inf,rto_cnt_cnt_rec_cat_hour_6_12,...,rto_sum_cnt_rec_cat_item_sales_level_16,rto_sum_cnt_rec_cat_item_sales_level_17,rto_sum_cnt_rec_cat_item_sales_level_2,rto_sum_cnt_rec_cat_item_sales_level_3,rto_sum_cnt_rec_cat_item_sales_level_4,rto_sum_cnt_rec_cat_item_sales_level_5,rto_sum_cnt_rec_cat_item_sales_level_6,rto_sum_cnt_rec_cat_item_sales_level_7,rto_sum_cnt_rec_cat_item_sales_level_8,rto_sum_cnt_rec_cat_item_sales_level_9
0,1000046842257508032:19_20,1,1,1,1,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
1,1000053191086897756:21_22,1,1,1,3,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667
2,1000077316672644068:22_23,1,1,1,1,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000107004384255968:24_25,1,1,1,1,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000143735450721424:22_23,1,1,1,1,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1000151384803367559:21_22,1,1,1,5,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,0.200000
6,1000161420494823356:18_19,1,1,1,1,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1000170383460699500:23_24,1,1,1,6,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,NaN,0.166667,NaN
8,1000192282597872986:23_24,1,1,1,2,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1000209033648236318:22_23,1,1,1,4,1.0,NaN,1.0,NaN,NaN,...,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cdr_rec = dfAll
cdr_dict = cdr_rec.to_dict(orient='records')
cdr_dict = [dict(list(x.items()) + [('TS', '~'.join(list(Discretization(x, TS_cat).values())[0]))] + [
    ('cat_str', cat_str(x, dict_cat))]) for x in cdr_dict]
cdr_rec = pd.DataFrame(cdr_dict).reset_index()
# one to many
cat_driver = pd.concat([pd.Series(row['index'], row['cat_str'].split(';'))
                        for _, row in cdr_rec.iterrows()]).reset_index()
cat_driver.columns = ['cat', 'index']
cdr_rec_m = pd.merge(cdr_rec, cat_driver, on='index', how='inner')
# Aggregation
if key_var=='':
    key_var_tmp = ['TS']
else:
    key_var_tmp = key_var+['TS']
agg_mst_m = _agg_df(cdr_rec_m, key_var_tmp+['cat'], sum_var, cnt_var, stat_var)
agg_mst_m['var_name'] = agg_mst_m[['method', 'variable', 'cat']].apply(
    lambda x: x[0] + '_' + x[1] + '_cat_' + str(x[2]), axis=1)
if key_var=='':
    agg_mst_m['key'] = agg_mst_m[key_var_tmp]
else:
    agg_mst_m['key'] = agg_mst_m[key_var_tmp].apply(lambda x: '{0}:{1}'.format(x[0], x[1]), axis=1)

In [11]:
agg_mst_m

,user_id,TS,cat,variable,value,method,var_name,key
0,24779788309075,18_19,cnt_rec_1,cnt_rec,3,sum,sum_cnt_rec_cat_cnt_rec_1,24779788309075:18_19
1,24779788309075,18_19,hour_20_Inf,cnt_rec,3,sum,sum_cnt_rec_cat_hour_20_Inf,24779788309075:18_19
2,24779788309075,18_19,item_brand_id_7838285046767229711,cnt_rec,3,sum,sum_cnt_rec_cat_item_brand_id_7838285046767229711,24779788309075:18_19
3,24779788309075,18_19,item_city_id_2022231041883668124,cnt_rec,1,sum,sum_cnt_rec_cat_item_city_id_2022231041883668124,24779788309075:18_19
4,24779788309075,18_19,item_city_id_5918626470536001929,cnt_rec,2,sum,sum_cnt_rec_cat_item_city_id_5918626470536001929,24779788309075:18_19
5,24779788309075,18_19,item_collected_level_11,cnt_rec,1,sum,sum_cnt_rec_cat_item_collected_level_11,24779788309075:18_19
6,24779788309075,18_19,item_collected_level_12,cnt_rec,2,sum,sum_cnt_rec_cat_item_collected_level_12,24779788309075:18_19
7,24779788309075,18_19,item_price_level_4,cnt_rec,1,sum,sum_cnt_rec_cat_item_price_level_4,24779788309075:18_19
8,24779788309075,18_19,item_price_level_5,cnt_rec,2,sum,sum_cnt_rec_cat_item_price_level_5,24779788309075:18_19
9,24779788309075,18_19,item_pv_level_14,cnt_rec,1,sum,sum_cnt_rec_cat_item_pv_level_14,24779788309075:18_19


In [12]:
agg_mst_m.pivot(index='key', columns='var_name', values='value')

ValueError: negative dimensions are not allowed

In [ ]:
df = pd.DataFrame({'A':[1,2,3],'B':['a;b','a;b','a;b;a']})

In [ ]:
df['B'].apply(lambda x:';'.join(sorted(set(str(x).split(';')))))

In [ ]:
dfTrain = pd.read_table(config.TRAIN_FILE,sep=' ')
dfTrain.drop_duplicates(inplace=True)
dfTrain.reset_index(inplace=True,drop =True)
dfTest = pd.read_table(config.TEST_FILE,sep=' ')

dfTrain = process(dfTrain)
dfTest = process(dfTest)

In [ ]:
dfAll = pd.concat([dfTrain,dfTest])
dfAll.reset_index(inplace=True,drop=True)
dfAll.shape

In [ ]:
Test = focus_one_record(dfAll,['user_id'])

In [ ]:
Test.dtypes

In [ ]:
dfTest['next_time_dur'].dt.days.max()

In [ ]:
dfTest['next_time_dur'].map(lambda x: x/np.timedelta64(1, 's'))

In [ ]:
dfTest.dtypes

In [ ]:
df = dfAll
key_var = ['user_id']

In [ ]:
pd.to_datetime(dfAll['time'])

In [ ]:
nameBase = '_'.join(key_var)
###当天前后的数据情况
df[nameBase+'_before_cnt'] = df.groupby(key_var+['day'])['context_timestamp'].rank(method='min')

In [ ]:
df.groupby('user_id')['context_timestamp'].rank(method='first')

In [ ]:
df.groupby('user_id')['context_timestamp'].rank(method='min')

In [ ]:
df.groupby(key_var+['day'])['context_timestamp'].count()

In [ ]:
key_var+['day']